In [24]:
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
data = open("inputs/17.input").read().strip()

In [18]:
data = """
x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504
""".strip()

In [26]:
lines = data.split("\n")
scans = [(l[0], *map(int, re.findall("\d+", l))) for l in lines]
max_x = max([v[3] for v in lines for v in scans if v[0] == "y"])
max_y = max([v[3] for v in lines for v in scans if v[0] == "x"])

# add a little padding
ground = np.empty(shape=(max_y + 10, max_x + 10), dtype="<U1")
ground[:, :] = "."
ground[0, 500] = "+"
for axis, a, b0, b1 in scans:
    if axis == "y":
        ground[a, b0:b1+1] = "#"
    else:
        ground[b0:b1+1, a] = "#"

ground_orig = ground.copy()

In [33]:
def show_ground(ground):
    print("View of ground from x=470...530")
    print("\n".join(["".join(g) for g in ground[:, 470:530]]))

In [37]:
ground = ground_orig.copy()

def propagate(y, x):
    # check if we have reached the bottom
    if y > ground.shape[0] - 2:
        return False
    
    # sand beneath us, just continue
    if ground[y+1, x] == ".":
        ground[y, x] = "|"
        propagate(y + 1, x)
    
    # there is already a waterstream beneath here from an earlier propagation
    elif ground[y+1, x] == "|":
        ground[y, x] = "|"
        return
    
    # hit a solid ground or steady water, propagate to the left and right
    if ground[y+1, x] in ["~", "#"]:
        # go left
        lx = x
        continue_left = False # will be False if we reach an end of the water stream, otherwise True if we need to continue downwards from the left
        while lx >= 1 and ground[y+1, lx-1] in ["~", "#"] and ground[y, lx-1] != "#": # as long as we are not stopped by clay on the left and have solid water or clay beneath us
            ground[y, lx] = "|"
            lx -= 1
        if lx >= 1 and ground[y, lx-1] == "#": # end of water stream here
            ground[y, lx] = "|"
        elif lx >= 1 and ground[y+1, lx-1] == ".": # 'open' bucket, water stream continues downwards
            ground[y, lx-1:lx+1] = "|"
            if not propagate(y, lx-1):
                continue_left = True
        
        # go right (equivalent to go left)
        continue_right = False
        rx = x
        while rx < ground.shape[1] - 1 and ground[y+1, rx+1] in ["~", "#"] and ground[y, rx+1] != "#":
            ground[y, rx] = "|"
            rx += 1
        if rx < ground.shape[1] - 1 and ground[y, rx+1] == "#":
            ground[y, rx] = "|"
        elif rx < ground.shape[1] - 1 and ground[y+1, rx+1] == ".":
            ground[y, rx:rx+2] = "|"
            if not propagate(y, rx+1):
                continue_right = True
        
        # now if we reached an end on the left and on the right, replace all | by still water ~
        if (lx >= 1) and (rx < ground.shape[1] - 1) and (not continue_left) and (not continue_right):
            ground[y, lx:rx+1] = "~"
            return True # tell the callee that the propagation led to still wate
    return False
        

propagate(1, 500)

show_ground(ground[:, :])

View of ground from x=470...530
..............................+.............................
..............................|.............................
..............................|.............................
..............................|.............................
.#............................|.............................
.#............................|.............................
.#............................|.............................
.#............................|.............................
.#............................|.............................
.#.....................#.#....|.............................
.#.....................#.#|||||||...........................
.#.....................#.#~~~~~#|...........................
.#...........#.........#.#~~~~~#|...........................
.#...........#.........#.#~~~~~#|...........................
.#...........#.........#.#~~~~~#|...........................
.#...........#.........#.#~~~~~#|....................

# Part 1

In [38]:
clay = ground=="#"
clay_ys, clay_xs = np.where(clay)
capped = ground[clay_ys.min():clay_ys.max()+1,:]
(capped == "|").sum() + (capped == "~").sum()

39162

# Part 2

In [39]:
(capped == "~").sum()

32047